<a href="https://colab.research.google.com/github/ellen107/CropsAndWeeds/blob/main/CropWeedClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v3 import models

In [2]:
!git clone https://github.com/ellen107/CropsAndWeeds.git


Cloning into 'CropsAndWeeds'...
remote: Enumerating objects: 734, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 734 (delta 8), reused 0 (delta 0), pack-reused 706
Receiving objects: 100% (734/734), 1.19 GiB | 42.44 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Updating files: 100% (1169/1169), done.


In [21]:
base_dir = '/content/CropsAndWeeds/output'
train_dir = '/content/CropsAndWeeds/output/train'
train_crop_dir = '/content/CropsAndWeeds/output/train/crops'
train_weed_dir = '/content/CropsAndWeeds/output/train/weeds'
test_dir = '/content/CropsAndWeeds/output/test'
test_crop_dir = '/content/CropsAndWeeds/output/test/crops'
test_weed_dir = '/content/CropsAndWeeds/output/test/weeds'
valid_dir = '/content/CropsAndWeeds/output/val'
valid_crop_dir = '/content/CropsAndWeeds/output/val/crops'
valid_weed_dir = '/content/CropsAndWeeds/output/val/weeds'

In [22]:
num_crop_train = len(os.listdir(train_crop_dir))
num_weed_train = len(os.listdir(train_weed_dir))
num_crop_validaition = len(os.listdir(valid_crop_dir))
num_weed_validation= len(os.listdir(valid_weed_dir))
num_crop_test = len(os.listdir(test_crop_dir))
num_weed_test= len(os.listdir(test_weed_dir))

In [23]:
print("Total Training Crop Images",num_crop_train)
print("Total Training Weed Images",num_weed_train)
print("--")
print("Total validation Crop Images",num_crop_validaition)
print("Total validation Weed Images",num_weed_validation)
print("--")
print("Total Test Crop Images", num_crop_test)
print("Total Test Weed Images",num_weed_test)
total_train = num_crop_train+num_weed_train
total_validation = num_crop_validaition+num_weed_validation
total_test = num_crop_test+num_weed_test
print("--")
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)

Total Training Crop Images 199
Total Training Weed Images 256
--
Total validation Crop Images 24
Total validation Weed Images 32
--
Total Test Crop Images 27
Total Test Weed Images 36
--
Total Training Images 455
--
Total Validation Images 56
--
Total Testing Images 63


In [24]:
IMG_SHAPE  = 224
batch_size = 32

In [25]:
image_gen_train = ImageDataGenerator(rescale = 1./255)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'binary')
image_generator_validation = ImageDataGenerator(rescale=1./255)
val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
directory=valid_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='binary')

Found 455 images belonging to 2 classes.
Found 56 images belonging to 2 classes.
Found 63 images belonging to 2 classes.


In [26]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [27]:
for layer in pre_trained_model.layers:
  print(layer.name)
  layer.trainable = False  #freeze the training layers of VGG-16

input_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [28]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(2, activation='sigmoid')(x)

In [29]:
x = tf.keras.layers.Dense(3, activation='softmax')(x)

In [30]:
model = tf.keras.Model(pre_trained_model.input,x)

In [31]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])

In [32]:
from keras.applications.mobilenet_v3 import models
vgg_classifier = model.fit(train_data_gen,
steps_per_epoch=(total_train//batch_size),
epochs = 5,validation_data=val_data_gen,
validation_steps=(total_validation//batch_size),
batch_size = batch_size,
verbose = 1)

UnidentifiedImageError: ignored